<a href="https://colab.research.google.com/github/ZeroLeon/NLP_Project_with_FastAI/blob/master/BBC_Summarization_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BBC News Summarization with BERT

## Use tools from this [repo](https://github.com/dmmiller612/bert-extractive-summarizer)

**Bert Extractive Summarizer by Derek Miller**

In [15]:
import os 
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive/fastai'
path = Path(base_dir +'/data/')
path.mkdir(parents=True,exist_ok=True) 
os.chdir(path)

Mounted at /content/gdrive


In [0]:
import glob

In [17]:
%cd BBC_Summarization

/content/gdrive/My Drive/fastai/data/BBC_Summarization


In [0]:
!pip install spacy==2.1.3
!pip install bert-extractive-summarizer

In [5]:
from summarizer import SingleModel

100%|██████████| 40155833/40155833 [00:01<00:00, 24303796.42B/s]


In [6]:
model = SingleModel('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 1138079.28B/s]


## Test on some text data

In [0]:
#%%capture
#!pip install plotly==4.0.0
#import plotly.express as px
from fastai.text import *
from fastai.callbacks import *

In [0]:
import spacy
import importlib
importlib.reload(spacy)

<module 'spacy' from '/usr/local/lib/python3.6/dist-packages/spacy/__init__.py'>

In [0]:


body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model = SingleModel('bert-base-uncased')
result = model(body, min_length=80)
full = ''.join(result)
print(full)

100%|██████████| 231508/231508 [00:00<00:00, 407507.65B/s]
17it [00:02,  5.66it/s]

The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. Officials with Tishman and RFR did not immediately respond to a request for comments. The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building. It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.


In [0]:
full

"The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The building sold fairly quickly after being publicly placed on the market only two months ago. The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building. Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest."

In [0]:
from pprint import pprint

In [0]:
pprint(full)

('The Chrysler Building, the famous art deco New York skyscraper, will be sold '
 'for a small fraction of its previous sales price. Officials with Tishman and '
 'RFR did not immediately respond to a request for comments. The incentive to '
 'sell the building at such a huge loss was due to the soaring rent the owners '
 'pay to Cooper Union, a New York college, for the land under the building. It '
 'is famous for its triangle-shaped, vaulted windows worked into the stylized '
 'crown, along with its distinctive eagle gargoyles near the top.')


## Implement on BBC News

### Prepare Data

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle --help

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        config {view, set, unset}
    competitions        Commands related to Kaggle competitions
    datasets            Commands related to Kaggle datasets
    kernels             Commands related to Kaggle kernels
    config              Configuration settings


In [0]:
!kaggle datasets download -d pariza/bbc-news-summary

  0% 0.00/3.75M [00:00<?, ?B/s]
100% 3.75M/3.75M [00:00<00:00, 62.6MB/s]


In [0]:
!unzip bbc-news-summary.zip

Archive:  bbc-news-summary.zip
  inflating: BBC News Summary.rar    


In [0]:
!unrar x BBC\ News\ Summary.rar

In [0]:
from glob import glob
import os

In [26]:
%cd BBC\ News\ Summary

/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary


In [0]:
# #ex
# pth = '/content/gdrive/My Drive/fastai/data/BBC_Summarization/'
# print(glob(pth+"*.txt"))

['/content/gdrive/My Drive/fastai/data/BBC_Summarization/327.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/098.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/244.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/510.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/172.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/456.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/353.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/205.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/260.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/239.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/321.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/467.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/055.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/462.txt', '/content/gdrive/My Drive/fastai/data/BBC_Summarization/116.txt', '/content

### Politics Category

In [0]:
path_pol ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/politics/'
path_pol_sum ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/Summaries/politics/'

In [0]:
pol_files = []
pol_sumfiles = []

pol_files.extend(glob(path_pol+'*.txt'))
pol_sumfiles.extend(glob(path_pol_sum+'*.txt'))

In [68]:
pol_files[:3]

['/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/politics/001.txt',
 '/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/politics/002.txt',
 '/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/politics/003.txt']

In [69]:
len(pol_files)

417

In [0]:
#function to get summarization of text file in category 'politics'

def pol_summarize(num,ratio=0.2):
  with open(pol_files[num],'r') as f, open(pol_sumfiles[num],'r') as fs:
    content = f.read()
    content_sum = fs.read()
    result = model(content,ratio, min_length=50)
    print('------------------------------')
    print('-----ARTICLE------------------------------------------------')
    pprint(content)
    print('-----BERT_SUM------------------------------------------------')
    pprint(result)
    print('-----LABEL_SUM-----------------------------------------------')
    pprint(content_sum)


In [78]:
pol_summarize(15)

11it [00:01,  6.48it/s]

------------------------------
-----ARTICLE------------------------------------------------
("Crisis 'ahead in social sciences'\n"
 '\n'
 'A national body designed to tackle skills shortages in key subjects should '
 'be set up, a committee of MPs has said.\n'
 '\n'
 'There was an "absolute crisis" in the recruitment of statisticians and other '
 'social science experts, the Commons science and technology committee added. '
 '"Major problems" are expected over the next 10 years as the social science '
 'workforce continues to age, it said. The committee was reporting on the '
 'work, strategy and spending plans of the Economic Social Research Council.\n'
 '\n'
 'The council aims to promote and support strategic research for social '
 'science postgraduates and provide social scientists to meet the needs of the '
 'country. It also liaises with the government and industry over their social '
 'science needs. In its report on the council\'s work the committee said: "We '
 'are deeply con

### Tech Category

In [0]:
path_tech ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/tech/'
path_tech_sum ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/Summaries/tech/'

In [0]:
tech_files = []
tech_sumfiles = []

tech_files.extend(glob(path_tech+'*.txt'))
tech_sumfiles.extend(glob(path_tech_sum+'*.txt'))

In [23]:
tech_files[:2]

['/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/tech/274.txt',
 '/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/tech/099.txt']

In [0]:
#function to get summarization of text file in category 'politics'

def tech_summarize(num,ratio=0.2):
  with open(tech_files[num],'r') as f, open(tech_sumfiles[num],'r') as fs:
    content = f.read()
    content_sum = fs.read()
    result = model(content,ratio,min_length=50)
    print('------------------------------')
    print('-----ARTICLE------------------------------------------------')
    pprint(content)
    print('-----BERT_SUM------------------------------------------------')
    pprint(result)
    print('-----LABEL_SUM-----------------------------------------------')
    pprint(content_sum)
  return result,content_sum


In [93]:
result,content = tech_summarize(18)

20it [00:03,  6.39it/s]

------------------------------
-----ARTICLE------------------------------------------------
('Intel unveils laser breakthrough\n'
 '\n'
 'Intel has unveiled research that could mean data is soon being moved around '
 'chips at the speed of light.\n'
 '\n'
 'Scientists at Intel have overcome a fundamental problem that before now has '
 'prevented silicon being used to generate and amplify laser light. The '
 'breakthrough should make it easier to interconnect data networks with the '
 'chips that process the information. The Intel researchers said products '
 'exploiting the breakthrough should appear by the end of the decade.\n'
 '\n'
 '"We\'ve overcome a fundamental limit," said Dr Mario Paniccia, director of '
 "Intel's photonics technology lab. Writing in the journal Nature, Dr Paniccia "
 '- and colleagues Haisheng Rong, Richard Jones, Ansheng Liu, Oded Cohen, Dani '
 'Hak and Alexander Fang - show how they have made a continuous laser from the '
 'same material used to make comput

[Comparison Result](https://copyleaks.com/compare-embed/compare-two-files/a06add53-eb08-4c81-8849-e6455a1b1915/12429058/1/1?key=x1xLAFMILXBO7f6rTyx9)

![Results](https://i.ibb.co/sWqWR1w/Snip20190911-9.png)

In [88]:
result

'Intel unveils laser breakthrough\n\nIntel has unveiled research that could mean data is soon being moved around chips at the speed of light. Scientists at Intel have overcome a fundamental problem that before now has prevented silicon being used to generate and amplify laser light. We\'re trying to take our silicon competency in manufacturing and apply it to new areas," said Dr Paniccia. This is despite the fact that silicon is a much better amplifier of light pulses than the form of the material used in fibre optic cables. Dr Paniccia said the work was the one of several steps needed if silicon was to be used to make components that could carry and process light in the form of data pulses. "'

In [0]:
content

In [90]:
result2,content2 = tech_summarize(28)

14it [00:02,  4.48it/s]

------------------------------
-----ARTICLE------------------------------------------------
('Open source leaders slam patents\n'
 '\n'
 'The war of words between Microsoft and the open source movement heated up '
 'this week as Linux founder Linus Torvalds led an attack on software '
 'patents.\n'
 '\n'
 'In a panel discussion at a Linux summit in California Mr Torvalds said '
 'software patents were a problem for the open source movement. Mitchell '
 'Kapor, chairman of the Mozilla foundation, warned that Microsoft could use '
 'patent lawsuits in the future. Linux is a freely-available alternative to '
 "Microsoft's Windows. It relies on a community of programmers for its "
 'development and is based on open source principles, which allow others to '
 'use and modify it without having to pay licence fees. The attack on software '
 'patents comes at a time when IBM has made 500 of its patents freely '
 'available. Other companies are expected to follow suit.\n'
 '\n'
 'There are betw

[Comparison Results](https://copyleaks.com/compare-embed/compare-two-files/bde32bad-9f90-4695-b033-d15ba624a55a/12429237/1/1?key=OOHUH5Aq2kHL3DrEjhS3)

![Results](https://i.ibb.co/vjYPLtm/Snip20190911-10.png")

In [91]:
print(result2)
print(content2)

Open source leaders slam patents

The war of words between Microsoft and the open source movement heated up this week as Linux founder Linus Torvalds led an attack on software patents. Mitchell Kapor, chairman of the Mozilla foundation, warned that Microsoft could use patent lawsuits in the future. The industry respects the open source movement and in fact many of the members we represent use the open source system to develop software," "We think the important point to make is that companies should be able to choose between patents, copyrights and open source as to the treatment of their intellectual discoveries, and not be forced into using one or the other," she added.
In a panel discussion at a Linux summit in California Mr Torvalds said software patents were a problem for the open source movement.There are between 150,000 and 300,000 registered software patents in the US and open source developers argue that many should never have been granted."The industry respects the open source

In [99]:
result3,content3 = tech_summarize(28,ratio=0.5)

14it [00:02,  4.28it/s]

------------------------------
-----ARTICLE------------------------------------------------
('Open source leaders slam patents\n'
 '\n'
 'The war of words between Microsoft and the open source movement heated up '
 'this week as Linux founder Linus Torvalds led an attack on software '
 'patents.\n'
 '\n'
 'In a panel discussion at a Linux summit in California Mr Torvalds said '
 'software patents were a problem for the open source movement. Mitchell '
 'Kapor, chairman of the Mozilla foundation, warned that Microsoft could use '
 'patent lawsuits in the future. Linux is a freely-available alternative to '
 "Microsoft's Windows. It relies on a community of programmers for its "
 'development and is based on open source principles, which allow others to '
 'use and modify it without having to pay licence fees. The attack on software '
 'patents comes at a time when IBM has made 500 of its patents freely '
 'available. Other companies are expected to follow suit.\n'
 '\n'
 'There are betw

In [100]:
print(len(result3),len(content3))
print(result3)
print(content3)

1021 1054
Open source leaders slam patents

The war of words between Microsoft and the open source movement heated up this week as Linux founder Linus Torvalds led an attack on software patents. In a panel discussion at a Linux summit in California Mr Torvalds said software patents were a problem for the open source movement. Linux is a freely-available alternative to Microsoft's Windows. The attack on software patents comes at a time when IBM has made 500 of its patents freely available. This is a view corroborated by the UK Patent Office. " Some panel members are worried that Microsoft would issue a series of patent lawsuits in the future. " The industry respects the open source movement and in fact many of the members we represent use the open source system to develop software," "We think the important point to make is that companies should be able to choose between patents, copyrights and open source as to the treatment of their intellectual discoveries, and not be forced into usin

In [0]:
from pprint import pprint

In [28]:
result,content = tech_summarize(30)

11it [00:01,  5.54it/s]

------------------------------
-----ARTICLE------------------------------------------------
('Apple sues to stop product leaks\n'
 '\n'
 'Computer firm Apple has issued a lawsuit to prevent online leaks of '
 'information about future products.\n'
 '\n'
 'The lawsuit, against an unidentified individual, comes just weeks before the '
 'MacWorld conference in San Francisco, used to showcase new products. The '
 'complaint said an "unidentified individual... has recently misappropriated '
 'and disseminated confidential information". The lawsuit was filed with the '
 'Santa Clara California Superior Court. Apple is famously secretive about its '
 'future product launches while Apple users are equally famous for speculating '
 'about new technology from the company. Fans have speculated in recent weeks '
 'about the possibility of a new type of iPod being announced at the MacWorld '
 'conference.\n'
 '\n'
 'Apple said in the seven-page complaint, filed on 13 December, that it did '
 'not k

In [29]:
print(result)
print(content)

Apple sues to stop product leaks

Computer firm Apple has issued a lawsuit to prevent online leaks of information about future products. Apple is famously secretive about its future product launches while Apple users are equally famous for speculating about new technology from the company. The company said it would amend the complaint once they had discovered the names of those who had allegedly leaked information.
More people are becoming aware of online security issues but they have little confidence that companies are doing enough to counter the threats, said security firm RSA, which carried out the poll.The method it has developed makes use of the chip embedded in bank cards and a special card reader which can generate unique codes that are active for a specified amount of time.Activcard is just one of the many companies, like RSA Security, which has been trying to come up with just that.Almost 70% of those asked in a poll said that net firms are not doing enough to protect people.

In [0]:
path_tech ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/business/'
path_tech_sum ='/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/Summaries/business/'

In [0]:
tech_files = []
tech_sumfiles = []

tech_files.extend(glob(path_tech+'*.txt'))
tech_sumfiles.extend(glob(path_tech_sum+'*.txt'))
tech_files.sort()
tech_sumfiles.sort()

In [59]:
tech_files[:2]

['/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/business/001.txt',
 '/content/gdrive/My Drive/fastai/data/BBC_Summarization/BBC News Summary/News Articles/business/002.txt']

In [0]:
#function to get summarization of text file in category 'politics'

def tech_summarize(num,ratio=0.5):
  with open(tech_files[num],'r') as f, open(tech_sumfiles[num],'r') as fs:
    content = f.read()
    content_sum = fs.read()
    result = model(content,ratio,min_length=50)
    print('------------------------------')
    print('-----ARTICLE------------------------------------------------')
    pprint(content)
    print('-----BERT_SUM------------------------------------------------')
    pprint(result)
    print('-----LABEL_SUM-----------------------------------------------')
    pprint(content_sum)
  return result,content_sum


In [61]:
a,b =   tech_summarize(5)

9it [00:01,  6.15it/s]


------------------------------
-----ARTICLE------------------------------------------------
('Japan narrowly escapes recession\n'
 '\n'
 "Japan's economy teetered on the brink of a technical recession in the three "
 'months to September, figures show.\n'
 '\n'
 'Revised figures indicated growth of just 0.1% - and a similar-sized '
 'contraction in the previous quarter. On an annual basis, the data suggests '
 'annual growth of just 0.2%, suggesting a much more hesitant recovery than '
 'had previously been thought. A common technical definition of a recession is '
 'two successive quarters of negative growth.\n'
 '\n'
 'The government was keen to play down the worrying implications of the data. '
 '"I maintain the view that Japan\'s economy remains in a minor adjustment '
 'phase in an upward climb, and we will monitor developments carefully," said '
 'economy minister Heizo Takenaka. But in the face of the strengthening yen '
 'making exports less competitive and indications of weake

In [62]:
a,b =   tech_summarize(100)

16it [00:02,  6.02it/s]


------------------------------
-----ARTICLE------------------------------------------------
('Australia rates at four year high\n'
 '\n'
 'Australia is raising its benchmark interest rate to its highest level in '
 "four years despite signs of a slowdown in the country's economy.\n"
 '\n'
 'The Reserve Bank of Australia lifted interest rates 0.25% to 5.5%, their '
 'first upwards move in more than a year. However, shortly after the Bank made '
 'its decision, new figures showed a fall in economic growth in the last '
 'quarter. The Bank said it had acted to curb inflation but the move was '
 'criticised by some analysts.\n'
 '\n'
 'The rate hike was the first since December 2003 and had been well-flagged in '
 'advance. However, opposition parties and some analysts said the move was '
 'ill-timed given data showing the Australian economy grew just 0.1% between '
 'October and December and 1.5% on an annual basis.\n'
 '\n'
 'The figures, representing a decline from the 0.2% growth in GD

## Explore what happened behind the scene

In [0]:
!pip install pytorch-transformers

In [0]:
from pytorch_transformers import BertModel,BertTokenizer

In [107]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 413971.71B/s]


In [104]:
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1)
          )
        )
        (in

In [111]:
example_txt = 'The war of words between Microsoft and the open source movement heated up this week as Linux founder Linus Torvalds led an attack on software patents.'
tokens = tokenizer.tokenize(example_txt)
tokenizer.tokenize(example_txt)

['the',
 'war',
 'of',
 'words',
 'between',
 'microsoft',
 'and',
 'the',
 'open',
 'source',
 'movement',
 'heated',
 'up',
 'this',
 'week',
 'as',
 'linux',
 'founder',
 'lin',
 '##us',
 'tor',
 '##val',
 '##ds',
 'led',
 'an',
 'attack',
 'on',
 'software',
 'patents',
 '.']

In [113]:
#This whole process is exactly the same with tokenizer.encode(example_txt)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokenizer.convert_tokens_to_ids(tokens)

[1996,
 2162,
 1997,
 2616,
 2090,
 7513,
 1998,
 1996,
 2330,
 3120,
 2929,
 9685,
 2039,
 2023,
 2733,
 2004,
 11603,
 3910,
 11409,
 2271,
 17153,
 10175,
 5104,
 2419,
 2019,
 2886,
 2006,
 4007,
 13979,
 1012]

In [116]:
tokens_tensor = torch.tensor([indexed_tokens])
torch.tensor([indexed_tokens])

tensor([[ 1996,  2162,  1997,  2616,  2090,  7513,  1998,  1996,  2330,  3120,
          2929,  9685,  2039,  2023,  2733,  2004, 11603,  3910, 11409,  2271,
         17153, 10175,  5104,  2419,  2019,  2886,  2006,  4007, 13979,  1012]])

In [0]:
#input tensor to the model
#Models from pytorch-transformer can return full list of hidden-states & attentions weights at each layer
bert_model(tokens_tensor)

In [0]:
#Extract embeddings from bert_model
def extract_embeddings(self, text: str, hidden: int=-2, squeeze: bool=False, reduce_option: str ='mean') -> ndarray:
        tokens_tensor = self.tokenize_input(text)
        hidden_states, pooled = self.model(tokens_tensor)

        if hidden < -1 and hidden > -12:
            if reduce_option == 'max':
                pooled = hidden_states[hidden].max(dim=1)
            elif reduce_option == 'median':
                pooled = hidden_states[hidden].median(dim=1)
            else:
                pooled = hidden_states[hidden].mean(dim=1)

        if squeeze:
            return pooled.detach().numpy().squeeze()

        return pooled


![Outputs](https://i.ibb.co/t8GmJKy/Snip20190911-11.png)

**Decomposition:**
* PCA


**Centroid-based Clustering:**
* Kmeans
* GaussianMixture

k depends on num of features and ratio of compression

**Can also try:**
* HDBSCAN (don't need to set k)
* others

**Neural coreference techniques**:
Check the official [README](https://github.com/huggingface/neuralcoref)

It's using SpaCy's tagger, parser and NER annotations to identify a set of potential coreference mentions.

# Next Step:

* Use RoBERTa as Source of Embeddings
* Try Different Clustering Algorithms
* Apply this Method on Chinese